### Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from masserstein import NMRSpectrum, estimate_proportions
import matplotlib.pyplot as plt
import pulp
import pickle

### Data

In [ ]:
experiment_name = 'Saccharose hydrolysis'

In [ ]:
mixture_paths = {'Saccharose hydrolysis': '../data/saccharose_hydrolysis/sucrose_better_baseline_constant_region_sizes.csv',
                'PMG 284 monitoring': '../data/PMG_284/284_edited_corrected.csv',
                'PMG 287 monitoring': '../data/PMG_287/preprocessed_mixture_whole.csv' 
                }
#(it's ok that we are using preprocessed_mixture for PMG 287 
#cause the only preprocessing here is zeros from edges i.e. taking region: (2.952301, 10.387961))

mixture_separators = {'Saccharose hydrolysis': '\t',
                'PMG 284 monitoring': '\t',
                'PMG 287 monitoring': '\t'
                }

python_integrals_paths = {'Saccharose hydrolysis': '../python_integrals/',
                        'PMG 284 monitoring': '../python_integrals/',
                        'PMG 287 monitoring': '../python_integrals/'}

substances_names = {'Saccharose hydrolysis': ['saccharose', 'alpha-glucose', 'beta-glucose', 'fructose'],
                    'PMG 284 monitoring': ['2-pentene', 'triethylsilane', 'products'],
                    'PMG 287 monitoring': ['1-hexene', 'triethylsilane', 'product']
                   }

integration_intervals = {'Saccharose hydrolysis': [(5.39173, 5.44305), (5.2178, 5.26134), 
                                                   (4.62026, 4.67207), (3.97917, 4.01542)],
                    'PMG 284 monitoring': [(5.3, 5.6), (3.6, 3.95),
                                           (1.16, 1.36)],
                    'PMG 287 monitoring': [(3.599926, 3.679897),
                                            (4.079348, 4.129027), 
                                            (4.668059, 4.857709), 
                                            (7.616258, 7.838666), 
                                            (8.440374, 8.647265),

                                            (3.343730, 3.406997),
                                            (3.756886, 3.832615), 
                                            (6.422073, 6.564104),

                                            (3.256018, 3.343730),
                                            (3.681636, 3.722376), 
                                            (3.722376, 3.756886),
                                            (3.994139, 4.078496)
                                          ]
                   }

#### Mixture in time

In [ ]:
mixture_time_data = pd.read_csv(mixture_paths[experiment_name], sep = mixture_separators[experiment_name])


In [ ]:
mixture_time_data = pd.read_csv(mixture_paths[experiment_name], sep = mixture_separators[experiment_name])
if experiment_name == 'Saccharose hydrolysis':
    ppm = mixture_time_data.iloc[:,0:1]
    ints = mixture_time_data.iloc[:,1:-1]
    mixture_time_data = pd.concat((ppm, ints), axis=1)
elif experiment_name == 'PMG 284 monitoring':
    ppm = mixture_time_data.iloc[:,:-2].iloc[:,0:1]
    ints = mixture_time_data.iloc[:,:-2].iloc[:,1:]
    mixture_time_data = pd.concat((ppm, ints), axis=1)
elif experiment_name == 'PMG 287 monitoring':
    ppm = mixture_time_data.iloc[:,:-1].iloc[:,0:1]
    ints = mixture_time_data.iloc[:,:-1].iloc[:,1:]
    mixture_time_data = pd.concat((ppm, ints), axis=1)

In [ ]:
names = ['ppm'] + ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]

In [ ]:
mixture_time_data.columns = names

In [ ]:
def load_spectrum(mixture_time_data, moment_of_time):
    ppm = mixture_time_data['ppm']
    intensity = mixture_time_data['t'+str(moment_of_time)]
    sp = NMRSpectrum(confs = list(zip(ppm, intensity)))
    sp.trim_negative_intensities()
    sp.normalize()
    return sp

In [ ]:
#load_spectrum(mixture_time_data, 999).plot(profile=True)

Note that no baseline correction is needed for saccharose hydrolysis anymore!

### Integrals changing in time

In [ ]:
data_cut_to_intervals = []
for interval in integration_intervals[experiment_name]:
    data_in_interval = mixture_time_data[
                                        mixture_time_data['ppm'].apply(lambda x:
                                                                               x>interval[0] and x<interval[1])
                                        ]
    data_cut_to_intervals.append(data_in_interval)
    
if experiment_name == 'PMG 284 monitoring':
    data_cut_to_intervals[2] = data_cut_to_intervals[2].fillna(0.)

integrals_changing_in_time = []
    
for timepoint in ['t' + str(nb) for nb in range(1, mixture_time_data.shape[1])]:
    
    integrals_fixed_time = []
    
    for data_in_interval in data_cut_to_intervals:
        
        x_fixed_interval = data_in_interval['ppm']
        y_fixed_interval = data_in_interval[timepoint]
        
        new_int = np.trapz(y = y_fixed_interval, x = x_fixed_interval)
        integrals_fixed_time.append(new_int)
        
    integral_entire_spectrum = np.trapz(y = mixture_time_data[timepoint], x = mixture_time_data['ppm'])
    integrals_fixed_time.append(integral_entire_spectrum)
    
    
    integrals_changing_in_time.append(integrals_fixed_time)

In [ ]:
python_integrals = np.array(integrals_changing_in_time)

In [ ]:
if experiment_name == 'PMG 287 monitoring':
    hexene_integral = python_integrals[:,:5].sum(axis=1).reshape(-1,1)
    triethylsilane_integral = python_integrals[:,5:8].sum(axis=1).reshape(-1,1)
    product_integral = python_integrals[:,8:12].sum(axis=1).reshape(-1,1)
    whole_integral = python_integrals[:,12:13].sum(axis=1).reshape(-1,1)
    python_integrals = np.concatenate([hexene_integral, triethylsilane_integral, product_integral, whole_integral],
                                     axis=1)

### Figures

In [ ]:
for i in range(python_integrals.shape[1]-1):
    plt.plot(python_integrals[:,i] / python_integrals[:, :-1].sum(1), 'p')

### Saving results

In [ ]:
colnames = [name + ': ' + str(interval) for name, interval in zip(substances_names[experiment_name],
                                                                integration_intervals[experiment_name])
           ]

In [ ]:
python_integrals_df = pd.DataFrame(python_integrals, columns=colnames + ['whole_spectrum'])

In [ ]:
python_integrals_df.to_csv(python_integrals_paths[experiment_name]+
                           'python_integral_' + 
                           '_'.join(experiment_name.split()) + 
                           '.csv',
                          index=False)